# MAEEG

## Implementation of the MAEEG model from [this](https://arxiv.org/abs/2211.02625) paper by Apple

In [29]:
import sys
sys.path.insert(0, '../../smoking_ml')
from lib.utils import *
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [27]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [84]:
X, y = load_and_window_nursing_list_for_convolution([0])
X = X[:,:1,:] # get only x
X_train,X_dev,y_train,y_dev = train_test_split(X,y,test_size=.5,stratify=y)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=128,shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=128,shuffle=True)

In [126]:
class MAAEEG(nn.Module):
    def conv_layer(self, in_channels, out_channels, kernel_size=8, n_groups=4, p_dropout=0.2):
        return nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding='same'),
            nn.Dropout(p=p_dropout),
            nn.GroupNorm(n_groups, out_channels),
            nn.GELU()
        )   
         
    def get_im_mask_fn(self, n_channels, p_mask=0.25):
        def mask_fn(x):
            n_channels_mask = int(np.floor(p_mask*x.shape[0]))  # num of channels to mask based on p_mask (% of channels to mask)
            mask_channel_idxs = torch.randint(0, n_channels, size=(x.shape[0], n_channels_mask)) # get random idxs of channels to mask for each batch
            for i,mask_channel_idx in enumerate(mask_channel_idxs): # for each random idxs (one for each batch)
                x[i, mask_channel_idx] = torch.randn(n_channels_mask, x.shape[2]) * np.sqrt(1/x.shape[1]) # set those channels to random normal values with std sqrt(1/n_channels)
            return x
        return mask_fn
    
    

    def __init__(self, in_channels):
        super().__init__()
        out_channels = 64
        self.six_layer_conv = nn.Sequential(
            self.conv_layer(in_channels= in_channels, out_channels=out_channels),
            self.conv_layer(in_channels=out_channels, out_channels=out_channels),
            self.conv_layer(in_channels=out_channels, out_channels=out_channels),
            self.conv_layer(in_channels=out_channels, out_channels=out_channels),
            self.conv_layer(in_channels=out_channels, out_channels=out_channels),
            self.conv_layer(in_channels=out_channels, out_channels=out_channels)
        )
        self.im_mask = self.get_im_mask_fn(n_channels=out_channels)

    def forward(self, x):
        ti = self.six_layer_conv(x)
        qi = self.im_mask(ti)
        return qi

model = MAAEEG(in_channels=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [127]:
for X,_ in trainloader:
    x = model(X)
    print(x.shape)
    break

torch.Size([128, 64, 101])
